In [1]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
    random.seed(seed)
    np.random.seed(seed)

In [ ]:
seed = 114514
set_seed(seed = seed)

In [1]:
!nvidia-smi

Fri Aug  5 11:41:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.78       Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   58C    P8     6W /  N/A |    603MiB /  6144MiB |     36%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
class Paddy(Dataset):
    def __init__(self, files_name, transform, mode = "train"):
        super(Paddy).__init__()
        self.tfm = transform
        self.mode = mode
        self.paddy_dict = {'bacterial_leaf_blight': 0, 'bacterial_leaf_streak': 1, 'bacterial_panicle_blight': 2, \
            'blast': 3, 'brown_spot': 4, 'dead_heart': 5, 'downy_mildew': 6, 'hispa': 7, 'normal': 8, 'tungro': 9}
        self.files_name = files_name

    def __len__(self):
        return len(self.files_name)
    
    def __getitem__(self, idx):
        file_name = self.files_name[idx]
        im = self.tfm(Image.open(file_name))
        if self.mode == "train":
            label = self.paddy_dict[file_name.split("/")[-2]]
        else:
            label = -1
        return im, label

In [5]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(180),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [6]:
### Configuration
batch_size = 64
n_epochs = 120
learning_rate = 0.001
K_fold = 5
early_stopping_count = 24
best_model_save_path = "./models"

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cuda


In [7]:
### Train_path is "paddy-disease-classification"; Test_path is "test_images"
Train_path = './'
Test_path = './test_images'

train_files_name = []
train_info = pd.read_csv(f'{Train_path}train.csv')
for index, row in train_info.iterrows():
    train_files_name.append(f'{Train_path}train_images/{row["label"]}/{row["image_id"]}')
random.shuffle(train_files_name)
    
test_files_name = []
for image in os.listdir(Test_path):
    test_files_name.append(f'{Test_path}/{image}')

In [8]:
Train_set = []
Val_set = []
set_num = len(train_files_name) // K_fold
for i in range(K_fold):
    Train_set.append(train_files_name[: i * set_num] + train_files_name[(i + 1) * set_num :])
    Val_set.append(train_files_name[i * set_num : (i + 1) * set_num])

In [20]:
def train():
    for i in range(K_fold):
        print(f"###################### Processing fold: {i} ###################### \n")
        model = models.resnet34(pretrained = True)
        model.fc = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(model.fc.in_features, 10)
        )
        model = model.to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.00001)
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 12, T_mult = 1)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr = 1e-5, max_lr = 0.001, step_size_up = 4, mode="triangular2")
        criterion = nn.CrossEntropyLoss()

        train_data = Paddy(files_name = Train_set[i], transform = train_transform, mode = "train")
        val_data = Paddy(files_name = Val_set[i], transform = test_transform, mode = "train")

        train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers = 4, pin_memory = True)
        val_loader = DataLoader(val_data, batch_size = batch_size, shuffle = True, num_workers = 4, pin_memory = True)
        
        with open('./record.txt', 'a') as f:
            f.write(f'Fold: {i + 1}\n')

        Acc = 0.
        stopping_count = 0
        for epoch in range(n_epochs):
            train_loss = []
            train_acc = []
            val_loss = []
            val_acc = []

            model.train()
            pbar = tqdm(train_loader, ncols = 120)

            for images, labels in pbar:
                optimizer.zero_grad()

                images, labels = images.to(device), labels.to(device)

                prediction = model(images)
                loss = criterion(prediction, labels)
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), max_norm = 20)
                optimizer.step()

                train_loss.append(loss.detach().item())

                ### [batch_size * 10]
                train_acc.append((prediction.argmax(dim = 1).flatten() == labels.flatten()).float().mean().item())

                pbar.set_description(f"Processing train epoch [{epoch + 1} / {n_epochs}]")
                pbar.set_postfix({'Train loss': '{0:1.5f}'.format(sum(train_loss) / len(train_loss)), \
                                  'Train accuracy': '{0:1.5f}'.format(sum(train_acc) / len(train_acc))})    

            scheduler.step()


            ### Validation Part
            model.eval()
            pbar = tqdm(val_loader, ncols = 120)

            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)

                with torch.no_grad():
                    prediction = model(images)
                    loss = criterion(prediction, labels)

                val_loss.append(loss.detach().item())

                ### [batch_size * 10]
                val_acc.append((prediction.argmax(dim = 1).flatten() == labels.flatten()).float().mean().item())

                pbar.set_description(f"Processing validation epoch [{epoch + 1} / {n_epochs}]")
                pbar.set_postfix({'Val loss': '{0:1.5f}'.format(sum(val_loss) / len(val_loss)), \
                                  'Val accuracy': '{0:1.5f}'.format(sum(val_acc) / len(val_acc))})

            Val_acc = sum(val_acc) / len(val_acc)

            with open('./record.txt', 'a') as f:
                f.write(f"Processing train epoch [{epoch + 1} / {n_epochs}]\n")
                f.write(f'Train loss: {sum(train_loss) / len(train_loss)}\n')
                f.write(f'Train acc: {sum(train_acc) / len(train_acc)}\n')
                f.write(f'Val loss: {sum(val_loss) / len(val_loss)}\n')
                f.write(f'Val acc: {sum(val_acc) / len(val_acc)}\n')

            if Val_acc > Acc:
                Acc = Val_acc
                print(f'Find the best model in Fold {i + 1}')
                model_path = best_model_save_path + 'Fold_' + str(i + 1) + '.ckpt'
                torch.save(model.state_dict(), model_path)
                stopping_count = 0
            else:
                stopping_count += 1
                if stopping_count >= early_stopping_count:
                    print("###################### Early stopped ###################### \n")
                    break

In [ ]:
train()

###################### Processing fold: 0 ###################### 



Processing validation epoch [1 / 120]: 100%|████| 33/33 [00:12<00:00,  2.61it/s, Val loss=1.38126, Val accuracy=0.58939]


Find the best model in Fold 1


Processing validation epoch [2 / 120]: 100%|████| 33/33 [00:06<00:00,  5.10it/s, Val loss=0.79601, Val accuracy=0.74277]


Find the best model in Fold 1


...

Processing validation epoch [22 / 120]: 100%|███| 33/33 [00:06<00:00,  5.06it/s, Val loss=0.10423, Val accuracy=0.97117]


Find the best model in Fold 2


Processing validation epoch [23 / 120]: 100%|███| 33/33 [00:06<00:00,  5.02it/s, Val loss=0.10373, Val accuracy=0.97159]


Find the best model in Fold 2


Processing train epoch [31 / 120]:  63%|█▉ | 82/131 [00:16<00:09,  4.94it/s, Train loss=0.07447, Train accuracy=0.97428]

In [ ]:
def predict(loader, k_fold):
    predictions = []
    for i in range(k_fold):
        tmp_prediction = []

        model_path = best_model_save_path + 'Fold_' + str(i + 1) + '.ckpt'

        model = models.resnet34(pretrained = False)
        model.fc = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(model.fc.in_features, 10)
        )
        model.load_state_dict(torch.load(model_path))
        model = model.to(device)
        model.eval()

        pbar = tqdm(loader, ncols = 120)
        for image, _ in pbar:
            image = image.to(device)
            with torch.no_grad():
                prediction = model(image)
                tmp_prediction.append(prediction.detach().cpu())
            pbar.set_description(f"###################### Processing test fold {i + 1} ######################")

        predictions.append(torch.cat(tmp_prediction, dim = 0))

    return sum(predictions)

In [ ]:
def test():
    augmented_loaders = []
    test_data = Paddy(files_name = test_files_name, transform = test_transform, mode = "test")
    test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    ### Test time augmentation
    augmented_loaders.append(test_loader)
    for _ in range(5):
        augmented_data = Paddy(files_name = test_files_name, transform = train_transform, mode = "test")
        augmented_loaders.append(DataLoader(augmented_data, batch_size = batch_size, shuffle = False, num_workers = 2, pin_memory = True))
        
    predictions = []
    for _, loader in enumerate(augmented_loaders):
        prediction = predict(loader, K_fold)
        predictions.append(prediction)

    augmented_prediction = 0.5 * predictions[0]
    for i in range(1, len(predictions)):
        augmented_prediction += 0.1 * predictions[i]
    
    augmented_pred = augmented_prediction.argmax(dim = 1)

    print(augmented_pred)
    print(np.unique(augmented_pred))

    return augmented_pred

In [ ]:
paddy_dict = {0: 'bacterial_leaf_blight', 
              1: 'bacterial_leaf_streak', 
              2: 'bacterial_panicle_blight',
              3: 'blast', 
              4: 'brown_spot', 
              5: 'dead_heart', 
              6: 'downy_mildew', 
              7: 'hispa', 
              8: 'normal', 
              9: 'tungro'
              }

augmented_pred = test()
print(f'Shape of the prediction is: {augmented_pred.shape}')

In [ ]:
test_path = os.listdir(Test_path)
with open('./submission.csv', 'w') as f:
    f.write('image_id,label\n')
    for id, label in enumerate(augmented_pred):
        f.write(f'{test_path[id]},{paddy_dict[label.item()]}\n')